# Dataset


## Dummy Loader


In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from data.dummy_dataset import DummyDataset, DummySimpleDataset

In [ ]:
dummy_simple_dataset = DummySimpleDataset(
    "val",
    3,
    (256, 256),
    max_items=100,
    seed=0,
    split_val_size=0.2,
    split_test_size=0.2,
    dataset_name="Dummy",
)

print(len(dummy_simple_dataset))
img, msk, name, _ = dummy_simple_dataset[79]

print(
    name,
    img.shape,
    img.dtype,
    img.min(),
    img.max(),
    msk.shape,
    msk.dtype,
    torch.unique(msk),
)

plt.imshow(np.moveaxis(img.numpy(), 0, -1))
# plt.imshow(msk.numpy())

In [ ]:
dummy_dataset = DummyDataset(
    "train",
    3,
    (256, 256),
    max_items=100,
    seed=0,
    split_val_size=0.2,
    split_test_size=0.2,
    dataset_name="Dummy",
    # shot_options=[1, 5, 10, 20],
    shot_options="all",
    sparsity_options=[
        ("point", [1, 5, 10, 20]),
        ("grid", (10, 20)),
        ("contour", "random"),
        ("skeleton", (0.1, 0.5)),
        # ("region", 0.5),
    ],
    # shot_sparsity_permutation=True,
    num_iterations=1.0,
    query_batch_size=5,
    split_query_size=0.9,
)

print(len(dummy_dataset))
support, query, _ = dummy_dataset[0]

supp_img, supp_msk, supp_name, supp_sparsity = support
qry_img, qry_msk, qry_name = query

print(
    supp_img.shape,
    supp_img.dtype,
    supp_img.min(),
    supp_img.max(),
    supp_msk.shape,
    supp_msk.dtype,
    torch.unique(supp_msk),
)
print(supp_name, supp_sparsity)
print(
    qry_img.shape,
    qry_img.dtype,
    qry_img.min(),
    qry_img.max(),
    qry_msk.shape,
    qry_msk.dtype,
    torch.unique(qry_msk),
)
print(qry_name)
print()

# plt.imshow(supp_msk[0].numpy())
# plt.imshow(qry_msk[0].numpy())

In [ ]:
print(dummy_dataset.support_batches)
for i in range(len(dummy_dataset)):
    support, query, _ = dummy_dataset[i]
    supp_img, supp_msk, supp_name, supp_sparsity = support
    qry_img, qry_msk, qry_name = query
    print(
        supp_img.shape[0],
        supp_msk.shape[0],
        len(supp_name),
        supp_sparsity,
        qry_img.shape[0],
        qry_msk.shape[0],
        len(qry_name),
    )

## Other


In [ ]:
from tasks.optic_disc_cup.datasets import RimOneDataset


rim_one = RimOneDataset(
    "train",
    3,
    (256, 256),
    max_items=100,
    seed=0,
    split_val_size=0.2,
    split_test_size=0.2,
    dataset_name="RIM-ONE DL",
    shot_options=[1, 5, 10, 20],
    sparsity_options=[
        ("point", [1, 5, 10, 20]),
        ("grid", [20, 10]),
        ("contour", [0.1, 0.5, 1]),
    ],
    shot_sparsity_permutation=True,
    query_batch_size=2,
    split_query_size=0.5,
)

# for i in range(rim_one.num_iterations):
#     print(rim_one.support_batches[i], rim_one.support_sparsities[i])


# Utilities


## Progress Bar


In [ ]:
import time
from tqdm import notebook

for i in notebook.trange(100):
    time.sleep(0.05)

for i in notebook.trange(100):
    time.sleep(0.05)


## Metric


In [ ]:
import torch

from learners.metrics import CustomMetric


metric = CustomMetric()

metric(torch.tensor([1, 0, 1]), torch.tensor([1, 0, 1]))

metric.compute()


## Utils


In [ ]:
from utils.utils import diff_dict


diff_dict(
    {
        "a": {"a": {"p": 1}},
        "b": 2,
        "p": (12, 34),
        "d": {"e": 4, "f": 5, "g": [6, 7, 8]},
        "g": [6, 7, 8],
    },
    {
        "a": {"a": {"p": 1, "q": 0}},
        "c": 3,
        "d": {"e": 4, "f": 5, "g": [6, 9, 8, {"a": 1}]},
        "g": [6, 9],
        "p": (12, 33, 34),
    },
)


In [ ]:
import os
import datetime


for item in os.listdir("logs/SL"):
    mtime = os.path.getmtime(os.path.join("logs/SL", item))
    print(datetime.datetime.fromtimestamp(mtime).isoformat())

## WandB


In [ ]:
import wandb

from config.constants import WANDB_SETTINGS

prefix = WANDB_SETTINGS["entity"] + "/" + WANDB_SETTINGS["project"] + "/"
wandb.Api().artifact(prefix + "wandb-history:v0")


In [ ]:
from utils.logging import prepare_ckpt_path_for_artifact


ckpt_path = "SL/2024-02-12 11-24/epoch=0 val_score=0.33.ckpt"

prepare_ckpt_path_for_artifact(ckpt_path)


In [ ]:
import wandb

from config.constants import WANDB_SETTINGS

from utils.wandb import wandb_login


def wandb_log_dataset_ref(
    dataset_path: str,
    dataset_name: str,
):
    wandb_login()
    wandb.init(
        tags=["helper"],
        project=WANDB_SETTINGS["project"],
        name=f"log dataset {dataset_name}",
    )
    dataset_artifact = wandb.Artifact(dataset_name, type="dataset")
    dataset_artifact.add_reference(f"file://{dataset_path}")
    wandb.log_artifact(dataset_artifact)
    wandb.finish()


wandb_log_dataset_ref("D:/Penelitian/FWS/Data/DRISHTI-GS", "DRISHTI")


# Other
